# Traitement des distances - repère absolu

Dans ce notebook, on définit des fonctions permettant de faire différents traitements sur les distances des atomes des molécules par rapport à quatre points fictifs (atomes fictifs) formant un repère orthonormé. Ces points sont placés en (0, 0, 0), (1, 0, 0), (0, 1, 0) et (0, 0, 1)


### Génération d'un tableau de coordonnées fictives

In [2]:
import numpy as np

atomes_nb = 6
atomes_nb_incl_fictif = atomes_nb + 1


# Génération d'une matrice de positions des atomes d'une molécule. Pour chaque atome, 
# une ligne contient 3 coordonnées x, y, z, chacune dans l'intervalle [-1,1[. 
def gen_pos(nb_atomes):
    return (np.random.random_sample(atomes_nb*3) * 2 - 1).reshape(-1,3)


positions = gen_pos(atomes_nb)
positions

array([[-0.05252438,  0.87234023,  0.46768813],
       [-0.32343911,  0.74620304,  0.95493684],
       [-0.10498419, -0.50055918,  0.47864919],
       [-0.08132486,  0.57630778,  0.61401783],
       [ 0.07518618,  0.87898277, -0.61667485],
       [-0.96647567,  0.16370443,  0.96868209]])

### Calcul de la matrice compressée de distances

Pour chaque atome de la molécule, la matrice compressée de distances contient la distance de l'atome avec les quatre points du repère absolu.

#### Calcul de la matrice de distances compressée

In [3]:
def matrice_distances_compr(positions):
    """ Renvoie la matrice de distances compressée des positions des atomes passées en paramètres
    La matrice de distances compressée est définie de la façon suivante : pour chaque atome, on calcule
    la distance avec chaque point du repère. Une ligne i de la matrice (n,4) correspond aux distances
    de l'atome i avec chacun des quatre points du repère"""
    
    nb_at = len(positions)
    
    # On renvoie un tableau vide si la molécule est vide (car sinon vstack lève une exception)
    if nb_at == 0:
        return []
    
    repere = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    repere = np.vstack([repere]*nb_at)

    positions = np.tile(positions, 4).reshape(4*nb_at, 3)
    
    return np.sqrt(np.sum(np.power(positions-repere, 2), 1)).reshape(nb_at, 4)


### Ajout du bruit

On définit ici une fonction prenant une matrice de coordonnées en entrée et renvoyant une matrice des coordonnées bruitées selon une loi normale de centre 0 et de variance 0,000833. Il s'agit d'une variance qui semble raisonnable par rapport aux déplacement que l'on applique aux atomes lorsqu'on optimise une molécule. En effet, cela revient à déplacer l'atome à une distance de moins de 0.05 angströms dans environ 68% des cas, et à une distance de moins de 0.1 angströms dans 95% des cas.

In [5]:
def positions_bruitees(positions):
    """ Ajoute du bruit à un tableau de positions et renvoie un tuple (positions bruitées, bruit)"""
    bruit = np.random.normal(loc=0.0, scale=0.028867, size=positions.shape)
    return ((positions + bruit), bruit)


(array([], dtype=float64), array([], dtype=float64))

#### Analyse statistique des différences de distances introduites par le bruitage pour tester sa conformité

On ajoute du bruit sur chaque coordonnée à l'origine du repère sur dix millions d'exemples et on analyse la distributions de distances obtenue. On constate que l'on obtient bien des résultats conformes à nos attentes.

In [85]:
import pandas as pd

origine = np.array([0, 0, 0])
origine = np.repeat(origine, 10000000).reshape(-1, 3)
pos_bruit = positions_bruitees(origine)

def calc_dist_origine(point):
    return math.sqrt(pow(point[0], 2) + pow(point[1], 2) + pow(point[2], 2))

distances = []

for pos in pos_bruit:
    distances.append(calc_dist_origine(pos))
    
distances = pd.Series(distances)
distances.describe(percentiles=[0.68, 0.95])

count    1.000000e+07
mean     4.607169e-02
std      1.943799e-02
min      1.683100e-04
50%      4.440512e-02
68%      5.405057e-02
95%      8.071034e-02
max      1.771635e-01
dtype: float64